In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

In [3]:
import os
import wandb
#os.environ['WANDB_MODE'] = 'dryrun'
wandb.init("sky-eye-full")
conf = wandb.config

In [4]:
from pprint import pprint
pprint(dict(conf))

{'amp_opt_level': 'O1',
 'aug_prob': 0.5,
 'batch_size': 2,
 'decoder_use_batchnorm': 'inplace',
 'encoder': 'efficientnet-b6',
 'epochs': 100,
 'loss_weights': {'focal': 4, 'jaccard': 1},
 'lr': 0.0005,
 'optim': 'adam',
 'scheduler_factor': 0.5,
 'scheduler_patience': 5,
 'segmentation_arch': 'Linknet',
 'training_resolution': 1024}


In [5]:
from glob import glob
train_dir = '/home/jovyan/work/datasets/xview/train'
test_dir = '/home/jovyan/work/datasets/xview/test'

In [6]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
import segmentation_models_pytorch as smp

In [7]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob)
])

In [8]:
from xv.nn.solaris.model_io import get_model
from xv.nn.nets import DownscaleLayer, XVNet
import segmentation_models_pytorch as smp

segmentation_types = {
    'PSPNet': smp.PSPNet,
    'FPN': smp.FPN,
    'Linknet': smp.Linknet,
    'Unet': smp.Unet
}

if 'pretrained_model' in dict(conf):
    model = get_model(conf.pretrained_model, 'torch', pretrained=conf.pretrained)
    preprocess_fn=None
else:
    model = segmentation_types[conf.segmentation_arch](conf.encoder,
                                                       classes=1,
                                                       activation='sigmoid')
    preprocess_fn = get_preprocessing_fn(conf.encoder)

model = model.cuda()

In [9]:
import random
from xv import dataset

random.seed(hash("😂"))


all_files = glob(f'{train_dir}/labels/*pre_disaster.json')
random.shuffle(all_files)

dev_ix = int(len(all_files)*.20)
dev_files = all_files[:dev_ix]
train_files = all_files[dev_ix:]

train_instances, dev_instances = dataset.get_instances(train_files), dataset.get_instances(dev_files)

len(train_instances), len(dev_instances)

(2240, 559)

In [10]:
train_dataset = dataset.BuildingSegmentationDataset(
    instances=train_instances,
    augment=augment,
    preprocess_fn=preprocess_fn,
)

dev_dataset = dataset.BuildingSegmentationDataset(
    instances=dev_instances,
    augment=None,
    preprocess_fn=preprocess_fn,
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
)

In [11]:
from xv.nn.losses import loss_dict, WeightedLoss
loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

In [12]:
import apex

optims = {
    'adam': torch.optim.Adam
}

optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [13]:
from apex import amp
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [14]:
wandb.watch(model);

In [15]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience)

In [16]:
best_score = 0
epoch = 0

In [ ]:
from xv import run
for epoch in range(epoch, conf.epochs):
    metrics = {'epoch': epoch}
    train_metrics = run.train_segment(model, optim, train_loader, loss)
    metrics.update(train_metrics)
    
    dev_metrics = run.evaluate_segment(model, dev_loader, loss)
    metrics.update(dev_metrics)
    wandb.log(metrics)
    
    score = metrics['building:f1']
    scheduler.step(-score)
    
    if score > best_score:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_score = score

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0



...

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0



In [ ]:
2

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))

util.vis_im_mask(image, masks['damage'], size=(512*2,512*2), opacity=.3);

In [ ]:
from collections import Counter
counts = Counter(len(i['pre']['features']) for i in train_dataset.instances)